# Mishnah-Powered AI Chatbot with Gemini on Kaggle

Following the model of the Torah-powered chatbot, this notebook creates a Retrieval-Augmented Generation (RAG) system that answers questions using the text of the Mishnah. It fetches all 63 tractates from the Sefaria API, indexes them using Google's Gemini models, and provides a user-friendly chat interface with Gradio.

## 1. Install Dependencies

We will install the latest LlamaIndex packages for Google GenAI integration, along with Gradio for the UI and other utility libraries.

In [ ]:
!pip install -U llama-index llama-index-llms-google-genai llama-index-embeddings-google-genai gradio requests pandas

## 2. Set Up Google API Key

We securely access your Google API Key from Kaggle Secrets. Ensure you have saved your key with the name `GOOGLE_API_KEY` in the **Add-ons > Secrets** menu.

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")
    print("API Key loaded successfully from Kaggle Secrets.")
except Exception as e:
    print(f"Error loading API key: {e}. Please ensure you have set the GOOGLE_API_KEY secret.")

## 3. Download the Mishnah from Sefaria

This section defines a `MishnahFetcher` class to download the text of all 63 tractates of the Mishnah. The Sefaria API uses a `Mishnah_{Tractate}` format for its endpoints.

In [ ]:
import requests
import pandas as pd

class MishnahFetcher:
    def __init__(self):
        # A list of all 63 tractates of the Mishnah
        self.mishnah_tractates = [
            'Berakhot', 'Peah', 'Demai', 'Kilayim', 'Sheviit', 'Terumot', 'Maasrot', 'Maaser Sheni', 'Challah', 'Orlah', 'Bikkurim',
            'Shabbat', 'Eruvin', 'Pesachim', 'Shekalim', 'Yoma', 'Sukkah', 'Beitzah', 'Rosh Hashanah', 'Taanit', 'Megillah', 'Moed Katan', 'Chagigah',
            'Yevamot', 'Ketubot', 'Nedarim', 'Nazir', 'Sotah', 'Gittin', 'Kiddushin',
            'Bava Kamma', 'Bava Metzia', 'Bava Batra', 'Sanhedrin', 'Makkot', 'Shevuot', 'Eduyot', 'Avodah Zarah', 'Avot', 'Horayot',
            'Zevachim', 'Menachot', 'Chullin', 'Bekhorot', 'Arakhin', 'Temurah', 'Keritot', 'Meilah', 'Tamid', 'Middot', 'Kinnim',
            'Kelim', 'Oholot', 'Negaim', 'Parah', 'Tohorot', 'Mikvaot', 'Niddah', 'Makhshirin', 'Zavim', 'Tevul Yom', 'Yadayim', 'Oktzin'
        ]

    def get_mishnah_tractate_text(self, tractate_name):
        # The API endpoint format is, e.g., 'Mishnah_Berakhot'
        api_tractate_name = f"Mishnah_{tractate_name}"
        url = f'https://www.sefaria.org/api/texts/{api_tractate_name}'
        print(f"Fetching {tractate_name}...")
        try:
            response = requests.get(url, params={'context': 0, 'commentary': 0, 'pad': 0, 'lang': 'en'})
            response.raise_for_status()
            data = response.json()
            return data.get('text', [])
        except requests.exceptions.RequestException as e:
            print(f"Could not fetch {tractate_name}: {e}")
            return []

    def get_all_mishnah_texts(self):
        all_texts = {}
        for tractate in self.mishnah_tractates:
            all_texts[tractate] = self.get_mishnah_tractate_text(tractate)
        return all_texts

# Fetch and process the text into a DataFrame
fetcher = MishnahFetcher()
texts = fetcher.get_all_mishnah_texts()

rows = []
for tractate, chapters in texts.items():
    if not chapters or not isinstance(chapters, list):
        print(f"Skipping {tractate} due to unexpected data format.")
        continue
    for chapter_idx, chapter in enumerate(chapters, 1):
        for mishnah_idx, mishnah in enumerate(chapter, 1):
            rows.append((tractate, chapter_idx, mishnah_idx, mishnah))

df_mishnah = pd.DataFrame(rows, columns=['tractate', 'chapter', 'mishnah', 'text'])
# Clean up HTML tags and remove empty rows
df_mishnah['text'] = df_mishnah['text'].astype(str).str.split("<").str[0]
df_mishnah.dropna(inplace=True)

print("\nMishnah text successfully downloaded and processed.")
df_mishnah.head()

## 4. Define Google GenAI LLM and Embedding Models

We initialize the models from the `google-genai` packages, which provide the latest integration with Google's services.

In [ ]:
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI

embedding_model = GoogleGenAIEmbedding()

llm = GoogleGenAI(
    model_name="models/gemini-pro",
    temperature=0,
)
print("Successfully initialized Google GenAI LLM and Embedding models.")

## 5. Configure LlamaIndex and Prepare Documents

We set the global LLM and embedding model for LlamaIndex. Then, we convert our DataFrame of mishnayot into a list of `Document` objects, grouping by chapter to ensure each document has sufficient context.

In [ ]:
from llama_index.core import Document, Settings

Settings.llm = llm
Settings.embed_model = embedding_model
Settings.chunk_size = 512

documents = []
for (tractate, chapter), group in df_mishnah.groupby(['tractate', 'chapter']):
    # Combine all mishnayot in a chapter into a single text block for better context
    chapter_text = "\n".join(f"Mishnah {r['mishnah']}: {r['text']}" for _, r in group.iterrows())
    
    metadata = {
        "tractate": tractate,
        "chapter": int(chapter)
    }
    documents.append(Document(text=chapter_text, metadata=metadata))

print(f"Created {len(documents)} chapter-level documents from the Mishnah.")

## 6. Build the Vector Store Index

The `VectorStoreIndex` is the core of our RAG system. It takes our documents, creates vector embeddings for them, and stores them in a way that allows for fast and accurate semantic retrieval.

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, show_progress=True)

## 7. Set Up the Chat Engine

We use the `ContextChatEngine`, which is ideal for RAG applications. It retrieves relevant text based on the user's query and uses it, along with a system prompt and conversation history, to formulate a grounded answer.

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import ContextChatEngine

# Define a system prompt for a Mishnah scholar
system_prompt = (
    "You are a Mishnah scholar assistant. Your role is to answer questions strictly based on the provided text from the Mishnah. "
    "Cite the tractate, chapter, and mishnah number for your answer. The context provided will be a full chapter; you must identify the specific mishnah within it. "
    "If the answer cannot be found in the provided text, you must respond with: 'I cannot find a teaching in the Mishnah to answer that.'"
)

# Create a retriever to search the index
retriever = index.as_retriever(similarity_top_k=10)

chat_memory = ChatMemoryBuffer.from_defaults(token_limit=8000)

# Create the chat engine
chat_engine = ContextChatEngine.from_defaults(
    retriever=retriever,
    memory=chat_memory,
    system_prompt=system_prompt
)

print("Successfully created Mishnah ContextChatEngine.")

## 8. Launch the Gradio Web Interface

Finally, we launch the Gradio `ChatInterface`. This creates a shareable public URL where anyone can interact with your Mishnah chatbot.

In [ ]:
import gradio as gr

def chat_interface(message, history):
    response = chat_engine.chat(message)
    return str(response)

gr.ChatInterface(
    chat_interface, 
    title="📖 Ask the Mishnah (with Gemini)",
    description="This chatbot answers questions using the 63 tractates of the Mishnah, powered by Google Gemini.",
    examples=[
        "From when may one recite the Shema in the evening?",
        "What are the four principal categories of damages?",
        "Who is wealthy?"
    ]
).launch(share=True)